[open @ Google colabo](https://colab.research.google.com/github/zaq9/case/blob/master/case20190107/case20190107.ipynb)

1月7日（月）の清算値で利益最大になるポジション（日経終値20038）

*   制約条件１：コスト100円以内 @1月4日（金）の清算値時点-
*  制約条件２：次のシナリオ全てでネットオプション価値が0以上

1.   日経21000円（IV28）
2.   日経19750円（IV35）
3.   日経19000円（IV40)
4.   日経18000円（IV70）


*   制約条件３:売りの総枚数20枚以内,各権利価格での買い枚数50枚以内
*   利用するオプションは1月限権利価格18000~21000のPUTとCALL




In [0]:
#for google colabo

!pip install pulp

In [0]:
import pandas as pd
import pulp as pu
from operator import itemgetter

In [0]:
def data(fname):
    base = 'https://raw.githubusercontent.com/zaq9/case/master/case20190107/csv_data/' 
    return pd.read_csv(base+ fname+'.csv',index_col=0)
  
data_init = data('data20190104')
op_names = data_init.index

In [0]:
data_init.head(5)

,PRICE
NAME,
C18000,1540
C18125,1420
C18250,1300
C18375,1190
C18500,1070


In [0]:
#変数定義　ops 買い枚数、ops_n 売り枚数　
ops = [pu.LpVariable(s, 0, 50, pu.LpInteger) for s in op_names] 
ops_n = [pu.LpVariable(s+'_n', 0, 20, pu.LpInteger) for s in op_names]

def seiyaku(read_df):
    return pu.lpSum(pu.lpDot(ops, read_df['PRICE'])) - pu.lpSum(pu.lpDot(ops_n, read_df['PRICE']))
  
  
m = pu.LpProblem('Max', sense=pu.LpMaximize)
m += pu.lpSum(ops_n) <= 20  # 合計売り枚数　２０枚以下
m += seiyaku(data_init) <= 100
m += seiyaku(data('data_nk18000_iv70') ) >=0
m += seiyaku(data('data_nk19000_iv40') ) >=0
m += seiyaku(data('data_nk19750_iv35') ) >=0
m += seiyaku(data('data_nk21000_iv28') ) >=0
m += seiyaku(data('data20190107') ) 

m.solve()
print(f" 最大値 = {m.objective.value()}")

ans = []
for op in ops:
    if op.value() != 0: ans.append([int(op.name[1:6] ) ,op.name, op.value()] )
    
for op in ops_n:
    if op.value() != 0: ans.append([int(op.name[1:6] ) ,op.name[0:6], -op.value()] )
    
ans.sort(key=itemgetter(0))

for x in ans :
    print(f"{x[1]}[{x[2]:.0f}]")


 最大値 = 38507.0
C18000[-10]
P18875[40]
P19000[31]
C19750[1]
C19875[50]
C20000[50]
C20125[50]
C20250[50]
C20375[50]
C20500[50]
C20625[50]
C20750[50]
C20875[50]
C21000[15]
P21000[-10]
